# NHPP - Wavenet for Regression

NHPP (Near History Past Points) Wavenet was meant to test the information flow through the Wavenet Time-Series predictor. It splits the dataset into two separate parts: one containing the 'older' datapoints, and one containing the more recent ones. The older datapoints are ran through the Wavenet and its result is appended to the more recent datapoints. This was supposed to give the last linear layer the most important datapoints for the prediction (which were regarded as being the most recent ones) without the information they possess being twisted by Wavenet. 

It was found that this method works on some datasets but not on others, (very slightly) improving or diminishing the accuracy (though with low probability of this being due to chance).

In [ ]:
# Importing modules

import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
from torch import nn, optim
from torch.autograd import Variable
from IPython.display import Audio
from scipy.io import wavfile
from collections import Counter
from collections import defaultdict
import math
import copy
import random
import pickle

In [ ]:
# Importing the data

bench_file = 'predictionsBenchmark'
tanke_bench = []
train_targets_bench = []
test_targets_bench = []
df = pd.read_csv(bench_file + '.csv')
months = []
days = []
weekdays = []
index = 0
value2index = defaultdict(int)
file = 'extendedBikeData'
for row in df.values:
  if (file in row[4]):
    
    if (math.isnan(row[3]) == False):
      tanke_bench.append(row[3])
      test_targets_bench.append(row[2])
      value2index[row[2]] = index
      
    elif(math.isnan(row[2]) == False):
      
      train_targets_bench.append(row[2])      
      test_targets_bench.append(row[2])
      value2index[row[2]] = index

    index += 1
    
    year = int(row[1].split('-')[0])
    month = int(row[1].split('-')[1])
    day = int(row[1].split('-')[2].split(' ')[0])
    date = datetime.date(year, month, day)
    months.append(month)
    days.append(day)
    weekdays.append(date.weekday())
#     print (date)
tanke_bench = tanke_bench[0:360] # Filtering for the predictions for which we can compare (the last 30 have no targets)
test_targets_bench = test_targets_bench[0:360] # Same here
print (len(tanke_bench))
train_targets_bench.extend(test_targets_bench)
vals = train_targets_bench
print (len(vals))

In [ ]:
# Rescaling the data
  
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

      
      
# Splitting dataset in train/test/valid datasets
test_set_size = 360
test_proportion = test_set_size/len(vals)
validation_proportion = 0.2

val_break = round(len(vals)*(1-(test_proportion + validation_proportion)))
test_break = round(len(vals)*(1-test_proportion))

values = vals[0 : val_break]
values_valid = vals[val_break : test_break]
values_test = vals[test_break:(len(vals))]
      
      
      
values_normed = torch.Tensor(vals)
values = torch.Tensor(values)
values_test = torch.Tensor(values_test)
values_valid = torch.Tensor(values_valid)



max_vals = max(vals)

values_normed = values_normed/ max_vals
values = values/max_vals
values_test = values_test/max_vals
values_valid = values_valid/max_vals


loss_fct = RMSELoss()

values_normed = values_normed.data.tolist()
values_test = values_test.data.tolist()
values = values.data.tolist()
values_valid = values_valid.data.tolist()


In [ ]:

# Function used to create the near history past points (nhpp for short)
def past_points_collector(values, nhpp_nbr):
  # Precomputing nhpp and additional features
  # nhpp = near history past points (i.e. past points introduced as features)
  # additional_features = days of the month (not one-hotted) + one-hot days of the week + one-hot months of the year

  
  if (additional_features == 50):
  
  
    # history = torch.FloatTensor(values).cuda()
    nhpp = []

    days_oh = list(np.zeros(31))
    days_oh[days[nhpp_nbr]] = 1
    nhpp.extend(days_oh)

    weekdays_oh = [0, 0, 0, 0, 0, 0, 0]
    weekdays_oh[weekdays[nhpp_nbr]] = 1
    nhpp.extend(weekdays_oh)

    months_oh = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    months_oh[months[nhpp_nbr]] = 1
    nhpp.extend(months_oh)

    nhpp.extend(values[0 : nhpp_nbr])


    nhpp = torch.FloatTensor(nhpp).unsqueeze(1).unsqueeze(0).cuda()

    for i in range (nhpp_nbr  + 1, len(values)):

      nhpp_temp = []
      days_oh = list(np.zeros(31))
      days_oh[days[i]-1] = 1
      nhpp_temp.extend(days_oh)

      weekdays_oh = [0, 0, 0, 0, 0, 0, 0]
      weekdays_oh[weekdays[i]] = 1
      nhpp_temp.extend(weekdays_oh)

      months_oh = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
      months_oh[months[nhpp_nbr]] = 1
      nhpp_temp.extend(months_oh)
      nhpp_temp.extend(values[i - nhpp_nbr : i])

      nhpp_temp = torch.FloatTensor(nhpp_temp).unsqueeze(1).unsqueeze(0).cuda()
      nhpp = torch.cat((nhpp, nhpp_temp), dim = 2)
  
  elif(additional_features == 0):
    
    nhpp = []
    
    if (nhpp_nbr > 0):
      nhpp.extend(values[0 : nhpp_nbr])
      nhpp = torch.FloatTensor(nhpp).unsqueeze(1).unsqueeze(0).cuda()
      for i in range (nhpp_nbr + 1, len(values)):

        nhpp_temp = []

        nhpp_temp.extend(values[i - nhpp_nbr : i])
        nhpp_temp = torch.FloatTensor(nhpp_temp).unsqueeze(1).unsqueeze(0).cuda()
        
        nhpp = torch.cat((nhpp, nhpp_temp), dim = 2)
    
    else:
      nhpp = torch.FloatTensor(nhpp).unsqueeze(1).unsqueeze(0).cuda()
      for i in range (0, len(values)):
        nhpp_temp = torch.FloatTensor([]).unsqueeze(1).unsqueeze(0).cuda()
        
        nhpp = torch.cat((nhpp,nhpp_temp), dim = 2)
      
    
    
  elif(additional_features == 3): # additional_features not one-hotted. Not recommended
    nhpp = [days[nhpp_nbr], weekdays[nhpp_nbr], months[nhpp_nbr]]
    if (nhpp_nbr > 0):
      nhpp.extend(values[0 : nhpp_nbr])
      nhpp = torch.FloatTensor(nhpp).unsqueeze(1).unsqueeze(0).cuda()

      for i in range (nhpp_nbr + 1, len(values)):
        nhpp_temp = [days[i], weekdays[i], months[i]]
        nhpp_temp.extend(values[i - nhpp_nbr : i])
        nhpp_temp = torch.FloatTensor(nhpp_temp).unsqueeze(1).unsqueeze(0).cuda()

        nhpp = torch.cat((nhpp, nhpp_temp), dim = 2)

    else:
      nhpp = torch.FloatTensor(nhpp).unsqueeze(1).unsqueeze(0).cuda()
      for i in range (0, len(values)):
        nhpp_temp = torch.FloatTensor([days[nhpp_nbr], 
                                       weekdays[nhpp_nbr], 
                                       months[nhpp_nbr]]).unsqueeze(1).unsqueeze(0).cuda()

#         print (nhpp.size(), nhpp_temp.size())
        nhpp = torch.cat((nhpp, nhpp_temp), dim = 2)
    

  return nhpp

In [ ]:
# The actual network
class WaveNet(nn.Module):
    def __init__(self, mu=1,n_residue=48, n_skip= 256, dilation_depth=10, 
                 n_repeat=5, nhpp = 50, nhpp_nbr = 0, additional_features = 0):
        # n_residue: residue channels
        # n_skip: skip channels
        # dilation_depth & n_repeat: dilation layer setup
        super(WaveNet, self).__init__()
        n_skip = int(np.ceil(n_skip/(dilation_depth*n_repeat))*(dilation_depth*n_repeat))
        self.nhpp_nbr = nhpp_nbr
        self.dilation_depth = dilation_depth
        dilations = self.dilations = [2**i for i in range(dilation_depth)] * n_repeat
        self.from_input = nn.Conv1d(in_channels=mu, out_channels=n_residue, kernel_size=1)
        self.conv_sigmoid = nn.ModuleList([nn.Conv1d(in_channels=n_residue, out_channels=n_residue, kernel_size=2, dilation=d)
                         for d in dilations])
        self.conv_tanh = nn.ModuleList([nn.Conv1d(in_channels=n_residue, out_channels=n_residue, kernel_size=2, dilation=d)
                         for d in dilations])
        self.skip_scale = nn.ModuleList([nn.Conv1d(in_channels=n_residue, out_channels=n_skip, kernel_size=1)
                         for d in dilations]) 
        self.residue_scale = nn.ModuleList([nn.Conv1d(in_channels=n_residue, out_channels=n_residue, kernel_size=1)
                         for d in dilations])
        self.conv_post_1 = nn.Conv1d(in_channels=n_skip, out_channels=n_skip, kernel_size=1)
        
        if (additional_features > 0):      
          self.linear1 = nn.Linear(in_features= n_skip + nhpp_nbr + additional_features, out_features=mu) # + 3, i.e. month, day, weekday
        else:
          self.linear1 = nn.Linear(in_features= n_skip + nhpp_nbr, out_features=mu)
          
          
        if (nhpp_nbr + additional_features > 0): 
          self.linear_nhpp = nn.Linear(in_features = nhpp_nbr + additional_features, 
                                         out_features = nhpp_nbr + additional_features)
          
        self.dropout = nn.Dropout (p=0.5)
        
    def forward(self, input):
        # Separating histories
        history, nhpp = input
        
        
        
        wn_history = self.preprocess(history) # Preparing the history for the Wavenet sequence
        ######### Wavenet Sequence ##########
        
        skip_connections = []
        loop_cst = 0
        # Convolution sequence
        for s, t, skip_scale, residue_scale in zip(self.conv_sigmoid, self.conv_tanh, self.skip_scale, self.residue_scale):
            wn_history, skip = self.residue_forward(wn_history, s, t, skip_scale, residue_scale)
            skip_connections.append(skip)
            loop_cst += 1
        # sum up skip connections
        wn_history = sum([s[:,:,-wn_history.size(2):] for s in skip_connections]) # If standard Wavenet
        ######## End of Wavenet Sequence #########
        # 
        output = self.postprocess(wn_history, nhpp)
        
        
        return output
    
    def preprocess(self, input):
        output = input.unsqueeze(0).unsqueeze(0).cuda()
        output = self.from_input(output)
        return output
    
    
    def postprocess(self, wn_history, nhpp):
        
        
            
        # Discard first nhpp (+1) points as we don't need them - they don't have nhpp history points
        wn_history = wn_history[:,:,-wn_history.size(2)+self.nhpp_nbr:]
        
        start_point = nhpp.size(1) - wn_history.size(2)
        
        nhpp = nhpp[:,start_point:].unsqueeze(0)
        
  
  
        # One layer to process the nhpp
        if (add_nhpp_FF == True):
          if (nhpp.size(1) + additional_features > 0):
            nhpp = torch.tanh(self.linear_nhpp(nhpp.transpose(1,2)).transpose(1,2))
        
        output = torch.cat((wn_history, nhpp), dim = 1)
        
        
        output = output.transpose(1,2)

#         Dropout
        if (add_dropout == True):
          if (training_so_dropout == True):
            output = self.dropout(output)
      
        output = self.linear1(output).squeeze(0).squeeze(1) # Last layer: linear - for regression
        
        
        return output
    
    def residue_forward(self, input, conv_sigmoid, conv_tanh, skip_scale, residue_scale):
        output = input
        output_sigmoid, output_tanh = conv_sigmoid(output), conv_tanh(output)
        output = torch.sigmoid(output_sigmoid) * torch.tanh(output_tanh)
        skip = skip_scale(output)
        output = residue_scale(output)
        output = output + input[:,:,-output.size(2):]
        return output, skip
    

    

In [ ]:
# Function which runs the algorithm

def run_algorithm_gpu(nhpp, n_residue, n_skip, dilation_depth, n_repeat,
                      max_epoch, batch_size, learn_rate, graph_cst, batches, additional_features = 0,
                      dbd_prediction = False, add_nhpp_FF = True, add_dropout = True):
    print ('nhpp', nhpp[nhpp_nbr].size(1))
    print ('additional features', additional_features)
    print ('n_residue', n_residue)
    print ('n_skip', n_skip)
    print ('dilation_depth', dilation_depth)
    print ('n_repeat', n_repeat)
    print ('max_epoch', max_epoch)
    print ('batch_size', batch_size)
    print ('learning rate', learn_rate)
    print ('add_nhpp_FF', add_nhpp_FF)
    print ('add_dropout', add_dropout)
    best_valid_epoch = 0
    net = WaveNet(mu=1,n_residue=n_residue,n_skip=n_skip,dilation_depth=dilation_depth,
                  n_repeat=n_repeat, nhpp = nhpp, nhpp_nbr = nhpp_nbr,
                 additional_features = additional_features).cuda()
    optimizer = optim.Adam(net.parameters(),lr=learn_rate)
    max_data = len(values)-batch_size
    
    
    loss_fct = RMSELoss()
    temp_loss = torch.Tensor(0)

    loss_save = [] 
    valid_loss_save = []
    valid_loss = 1000000
    last_loss = 1000000
    
    sequence_lowering_loss = 0
    
    for epoch in range(max_epoch):
        optimizer.zero_grad()
        loss = 0
        training_so_dropout = True
        for j in range(int(batches)):
            i = random.randint(0, len(values) - batch_size)
            batch = torch.FloatTensor(values[i : i + batch_size]).cuda()

            nhpp_sample = nhpp[nhpp_nbr][:,:, i : i + batch_size].cuda()
            x = (batch[:-1], nhpp_sample[0,:,:-1])
            logits = net(x)
            sz = logits.size(0)
            temp_loss = loss_fct(logits, batch[-sz:])
            
            loss = loss + temp_loss
         
        loss = loss/batches
        loss.backward()
        optimizer.step()
        loss_save.append(float(loss))
        if (epoch % graph_cst == 0):
          training_so_dropout = False
      
          if (dbd_prediction == False):
              batch = torch.FloatTensor(values_valid).cuda()
              nhpp_sample = nhpp_valid[nhpp_nbr]

              x = (batch[:-1], nhpp_sample[0,:,:-1])

              logits = net(x)
              l = logits 
              sz = logits.size(0)
              v_loss = loss_fct(logits, batch[-sz:])
          else:    
            diff, t_loss = test_generation(nt = net, nhpp = nhpp_test[nhpp_nbr], plot=False, prediction_window = 30)
            
          if (v_loss < valid_loss):
            best_valid_epoch = epoch
            valid_loss = v_loss
            best_net = WaveNet(mu=1,n_residue=n_residue,n_skip=n_skip,dilation_depth=dilation_depth,
                                n_repeat=n_repeat, nhpp = nhpp, nhpp_nbr = nhpp_nbr,
                                additional_features = additional_features).cuda()
            best_net.load_state_dict(copy.deepcopy(net.state_dict()))
            
            valid_loss_save.append(float(valid_loss))
            
     
    return best_net, loss_save, valid_loss_save, best_valid_epoch

In [ ]:
# Generating each point given its past (used for prediction of 30 points into the future)
# Then, generate the next point given its past (including the prediction at the previous step)
# Repeat for prediction_window turns.

additional_features = 50
def test_generation( first_points = 0,last_points = 0, 
                    prediction_window = 30, nt = None, 
                    nhpp = None, plot = False):
  # Finding the difference between the two 
  i = 0
  loss_fct = RMSELoss()
  nhpp_nbr = nhpp.size(1) - additional_features
  batch = torch.FloatTensor(values_test[i : (i+batch_size)]).cuda()
  nhpp_sample = nhpp[:,:, i : (i+batch_size)].cuda()

  x = (batch[:-1], nhpp_sample[0,:,:-1])
  
  logits = nt(x)
  
  
  start_point = batch.size(0) - logits.size(0)
  
  
  n = len(values_test)-start_point
  results = []
  for i in range (0,n-prediction_window+1, prediction_window):
    index = value2index[ values_test[i + prediction_window] ]
    batch_temp = values_test[i : (i + start_point + 1)]
    nhpp_temp = nhpp[:,:,i: (i + start_point + 1)].squeeze(0).transpose(0,1)
    
    for j in range (prediction_window):
      
      batch = torch.FloatTensor(batch_temp).cuda()
      
      nhpp_sample = nhpp_temp.clone()
      nhpp_sample = nhpp_sample.transpose(0,1).cuda()
      x = (batch[:-1], nhpp_sample[:,:-1])
      y = nt.forward(x)
      batch_temp = batch_temp[1:]
      batch_temp.append(torch.FloatTensor([float(y.data)]))
  
      features = []
      
      if (additional_features == 50):
  
        nhpp_temp = nhpp_temp[:][1:]
      
        days_oh = list(np.zeros(31))
        days_oh[days[index]] = 1
        features.extend(days_oh)

        weekdays_oh = [0, 0, 0, 0, 0, 0, 0]
        weekdays_oh[weekdays[index]] = 1
        features.extend(weekdays_oh)

        months_oh = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        months_oh[months[index]] = 1
        features.extend(months_oh)
        
        if nhpp_nbr > 0:
          features.extend(batch_temp[-nhpp_nbr:])

        features = torch.Tensor(features).cuda().unsqueeze(0)
        nhpp_temp = torch.cat((nhpp_temp, features), dim = 0)
        
        
      else:
        nhpp_temp = nhpp_temp[:][1:]
        
        if nhpp_nbr > 0:
          features.extend(batch_temp[-nhpp_nbr:])

        features = torch.Tensor(features).cuda().unsqueeze(0)
        nhpp_temp = torch.cat((nhpp_temp, features), dim = 0)
      
        
      results.append(float(y.data))
  targets2 = values_test[start_point:(start_point+(math.floor(n/prediction_window))*prediction_window)]
  
  loss2 = loss_fct (torch.FloatTensor(results), torch.FloatTensor(targets2))
  
  
  test_data_outside = (n-prediction_window+1) % prediction_window
  
  
  bench_loss = loss_fct(torch.Tensor(tanke_bench[start_point:(start_point+(math.floor(n/prediction_window))*prediction_window)]), 
                        torch.Tensor(test_targets_bench[start_point:(start_point+(math.floor(n/prediction_window))*prediction_window)]))
  
  diff = loss2 - bench_loss
  
    
    
  if (plot == True):
    print ('generation loss for nhpp = ', nhpp_nbr, ' is:', float(loss2.data.tolist()), 'while benchmark loss for window:', bench_loss)
    print ('generation - benchmark', loss2 - bench_loss)
    

    window_loss = loss_fct(torch.FloatTensor(results[first_points:last_points]), torch.FloatTensor(targets2[first_points:last_points]))
    plt.plot(results, ms = 0.1, color = 'green', label = 'predictions')
    plt.plot(tanke_bench[start_point:(start_point+(math.floor(n/prediction_window))*prediction_window)], ms = 0.1, label = 'benchmark')
    plt.plot(targets2, ms = 0.1, label = 'targets')
    plt.legend()
    plt.show()
  targets2 = torch.FloatTensor(targets2)
  results = torch.FloatTensor(results)
  return diff, float(loss2.data)

In [ ]:
# Applying algorithm to the data, takes a long time

additional_features = 50
prediction_train_size = 100

import time


n_skips = [1300]


learn_rates = [1e-3]

nhpp_nbrs = [0, 20]


dilation_depths = [4] 

training_so_dropout = True
add_nhpp_FF = False
add_dropout = False





# Constants

number_of_experiments = 5 #constant saying how many experiments should be run for one setting of hyperparameters
# it's used to calculate the mean standard deviation of the test loss in the case we want to make a comparison with 
# other models



max_epoch = 700
batches = 5
graph_cst = math.floor(max_epoch/50)
print ('graph cst', graph_cst)
# batch_size = len(values)
n_residue = 25
n_repeat = 2







calc_length = len(nhpp_nbrs)*len(dilation_depths)*len(n_skips)*len(learn_rates) * number_of_experiments
calc_count = 0
dbd_prediction = False




net = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(int))))
nhpp = defaultdict(int)
nhpp_test = defaultdict(int)
nhpp_valid = defaultdict(int)
losses = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(
                                lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(int))))))))


print (len(values))


for nhpp_nbr in nhpp_nbrs:
  for learn_rate in learn_rates:
    nhpp[nhpp_nbr] = past_points_collector(values = values, nhpp_nbr = nhpp_nbr)
    nhpp_test[nhpp_nbr] = past_points_collector(values_test, nhpp_nbr)
    nhpp_valid[nhpp_nbr] = past_points_collector(values_valid, nhpp_nbr)

    for n in range(len(n_skips)):
      
      n_skip = n_skips[n] - nhpp_nbr - additional_features

        #   for n_skip in n_skips:

      for dilation_depth in dilation_depths:
            batch_size = 2**(dilation_depth+1) + nhpp_nbr + prediction_train_size
            total_train_loss = torch.zeros(max_epoch)
            total_valid_loss = []
            best_valid_train_loss = []
            total_test_loss = []
    #     for nhpp_nbr in nhpps:


            for repeat in range (number_of_experiments):

              
              start = time.time()
              calc_count += 1


              net[nhpp_nbr][n_skip][dilation_depth][learn_rate], loss_save, valid_loss, best_valid_epoch = run_algorithm_gpu(nhpp = nhpp, 
                                                   n_residue = n_residue, 
                                                   n_skip = n_skip, 
                                                   dilation_depth = dilation_depth,  
                                                   n_repeat = n_repeat,
                                                   batch_size = batch_size, 
                                                   learn_rate = learn_rate, 
                                                   max_epoch = max_epoch,
                                                   graph_cst = graph_cst,
                                                   additional_features = additional_features,
                                                   dbd_prediction = dbd_prediction,
                                                   batches = batches,
                                                   add_nhpp_FF = add_nhpp_FF,
                                                   add_dropout = add_dropout)
          
              print ('best validation epoch', best_valid_epoch)
              
              
              # Calculating test loss
              batch = torch.FloatTensor(values_test).cuda()
              nhpp_sample = nhpp_test[nhpp_nbr]
              x = (batch[:-1], nhpp_sample[0,:,:-1])
              logits = net[nhpp_nbr][n_skip][dilation_depth][learn_rate](x)
              sz = logits.size(0)
              t_loss = loss_fct(logits, batch[-sz:])
              
              
              
          
              total_train_loss += torch.Tensor(loss_save)
              total_valid_loss.append(min(valid_loss))
              best_valid_train_loss.append(loss_save[best_valid_epoch])
              total_test_loss.append(float(t_loss))
              
              
              
              end = time.time()
              print ('---------------', 'done', (calc_count/calc_length) * 100, 
                    '% -----------------', 'time left', (end-start) * (calc_length-calc_count))
            
            
            total_train_loss = total_train_loss / number_of_experiments
            total_valid_loss = np.mean(total_valid_loss) 
            best_valid_train_loss = np.mean(best_valid_train_loss) 
#             total_test_loss = total_test_loss
            
            
            losses[nhpp_nbr][n_residue][n_skip][dilation_depth][n_repeat][prediction_train_size][learn_rate][max_epoch] = [total_train_loss, 
                                                                                                                np.mean(total_valid_loss),
                                                                                                                np.mean(best_valid_train_loss),
                                                                                                                np.mean(total_test_loss),
                                                                                                                np.std(total_test_loss)]
            
            test_generation(first_points = 0, last_points = 60, prediction_window = 30, 
                            nt = net[nhpp_nbr][n_skip][dilation_depth][learn_rate],
                            nhpp = nhpp_test[nhpp_nbr])

In [ ]:
class l_class:
  def __init__(self):

        self.nhpp_nbr = 0

        self.n_residue  = 0

        self.n_skip  = 0

        self.n_repeat   = 0
        
        self.prediction_train_size = 0
        
        self.dilation_depth = 0
        
        self.learn_rate = 0
        
        self.max_epoch = 0
        
        self.loss_train = []
        self.loss_train_zero = []
        
        self.loss_valid = 0
        self.loss_valid_zero = 0
        
        self.loss_test = 0
        self.loss_test_zero = 0
        
        self.best_valid_train_loss = 0
        self.best_valid_train_loss_zero = 0
        
        self.test_std = 0

In [ ]:
# Showing the results and deciding which ones are good and which ones are not
# write_file_title = ""
write_file_title = "xgboost_comparison_wavenet_" + file +  "_results.txt"

  
test_stds = []

f = open(write_file_title,"wb")
object_list = []

training_so_dropout = False
loss_fct = RMSELoss()
min_los = 10000
min_diff = 10000
los_list = []
for nhpp_nbr in losses.keys():
  for n_residue in losses[nhpp_nbr].keys():
    for n_skip in losses[nhpp_nbr][n_residue].keys():
      for dilation_depth in losses[nhpp_nbr][n_residue][n_skip].keys():
        for n_repeat in losses[nhpp_nbr][n_residue][n_skip][dilation_depth].keys():
          for prediction_train_size in losses[nhpp_nbr][n_residue][n_skip][dilation_depth][n_repeat].keys():
            for learn_rate in losses[nhpp_nbr][n_residue][n_skip][dilation_depth][n_repeat][prediction_train_size].keys():
              for max_epoch in losses[nhpp_nbr][n_residue][n_skip][dilation_depth][n_repeat][prediction_train_size][learn_rate].keys():
                  if( type(losses[nhpp_nbr][n_residue][n_skip][dilation_depth][n_repeat][prediction_train_size][learn_rate][max_epoch]) != int):
                    
                    # current results
                    los_train = losses[nhpp_nbr][n_residue][n_skip][dilation_depth][n_repeat][prediction_train_size][learn_rate][max_epoch][0]
                    los_valid = losses[nhpp_nbr][n_residue][n_skip][dilation_depth][n_repeat][prediction_train_size][learn_rate][max_epoch][1]
                    best_valid_train_loss = losses[nhpp_nbr][n_residue][n_skip][dilation_depth][n_repeat][prediction_train_size][learn_rate][max_epoch][2]
                    test_los = losses[nhpp_nbr][n_residue][n_skip][dilation_depth][n_repeat][prediction_train_size][learn_rate][max_epoch][3]
                    test_std = losses[nhpp_nbr][n_residue][n_skip][dilation_depth][n_repeat][prediction_train_size][learn_rate][max_epoch][4]

                    test_stds.append(test_std)
                    los_train_zero = losses[0][n_residue][n_skip+nhpp_nbr][dilation_depth][n_repeat][prediction_train_size][learn_rate][max_epoch][0]
                    los_valid_zero = losses[0][n_residue][n_skip+nhpp_nbr][dilation_depth][n_repeat][prediction_train_size][learn_rate][max_epoch][1]
                    best_valid_train_loss_zero = losses[0][n_residue][n_skip+nhpp_nbr][dilation_depth][n_repeat][prediction_train_size][learn_rate][max_epoch][2]
                    test_los_zero = losses[0][n_residue][n_skip+nhpp_nbr][dilation_depth][n_repeat][prediction_train_size][learn_rate][max_epoch][3]
                    test_std_zero = losses[0][n_residue][n_skip+nhpp_nbr][dilation_depth][n_repeat][prediction_train_size][learn_rate][max_epoch][4]

                    print ('nhpp', nhpp_nbr, 'n_skip', n_skip, 'dilation', dilation_depth, 
                         'valid diff',(los_valid_zero - los_valid)/los_valid_zero,
                         'train diff', (min(los_train_zero) - min(los_train))/min(los_train_zero), 
                        'test loss diff', float((test_los_zero - test_los) / test_los_zero ))
                    print ('los valid', los_valid, los_valid_zero)
                    print ('test_std and test_std_zero', test_std, test_std_zero)
                  

    
#                     batch = torch.FloatTensor(values_valid).cuda()
      
#                     nhpp_sample = nhpp_valid[nhpp_nbr]
#                     nhpp_sample_zero = nhpp_valid[0]

#                     x = (batch[:-1], nhpp_sample[0,:,:-1])
#                     x_zero = (batch[:-1], nhpp_sample_zero[0,:,:-1])

#                     logits = net[nhpp_nbr][n_skip][dilation_depth][learn_rate](x)
#                     logits_zero = net[0][n_skip+nhpp_nbr][dilation_depth][learn_rate](x_zero)
              
#                     sz = logits.size(0)
#                     sz_zero = logits_zero.size(0)
                  
#                     t_loss = loss_fct(logits, batch[-sz:])
#                     t_loss_zero = loss_fct(logits_zero, batch[-sz_zero:])
                    
#                     plt.plot(batch[-sz:].data.tolist(), label = 'targets')
#                     plt.plot(logits.data.tolist(), label = 'non_generated')
#                     plt.legend()
#                     print (t_loss, t_loss_zero)
                    
#                     diff, l = test_generation(first_points = 0,last_points = 0, 
#                     prediction_window = 30, nt = net[nhpp_nbr][n_skip][dilation_depth][learn_rate], 
#                     nhpp = nhpp_test[nhpp_nbr], plot = True)
                    
  
                    # Writing to file
                    object_to_write = l_class()
                    
                    object_to_write.nhpp_nbr      = nhpp_nbr

                    object_to_write.n_residue       = n_residue

                    object_to_write.n_skip      = n_skip

                    object_to_write.n_repeat   = n_repeat

                    object_to_write.prediction_train_size = n_repeat

                    object_to_write.learn_rate = learn_rate
                    
                    object_to_write.dilation_depth = dilation_depth

                    object_to_write.max_epoch = max_epoch
                    
                    object_to_write.loss_train = los_train
                    object_to_write.loss_train_zero = los_train_zero
                    
                    object_to_write.loss_valid = los_valid
                    object_to_write.loss_valid_zero = los_valid_zero
                    
                    object_to_write.loss_test = test_los
                    object_to_write.loss_test_zero = test_los_zero

                    object_to_write.best_valid_train_loss = best_valid_train_loss
                    object_to_write.best_valid_train_loss_zero = best_valid_train_loss_zero
                    
                    object_to_write.test_std = test_std
                    
                    
                    object_list.append(object_to_write)
                    
                    
#                     plt.show()
pickle.dump(object_list, f)
plt.hist(test_stds)
                    
    
#                     if (diff < min_diff):
#                       min_diff = diff
#                       best_nhpp = nhpp_nbr
#                       best_n_skip = n_skip
#                       best_dilation = dilation_depth
    
    
# #                     plt.plot(los_test, label = str(nhpp_nbr) + ' ' + str(n_skip) + ' ' + str(dilation_depth))       
# #                     plt.legend()
        
#                     los_list.append(l)
#                     if (l < min_los):
#                       min_los = l
#                       best_parameters = [nhpp_nbr, n_skip, dilation_depth, learn_rate]            


#                     print ('difference between generated and non generated', loss_fct(torch.Tensor(generated).cuda(), logits))
#                     plt.show()
  #                   plt.legend()
  #                   print (np.mean(los[1400:]), nhpp_nbr)
    #                 print (los)
    #                 print (nhpp_nbr)
f.close()